In [ ]:
ONLY_PUBLIC = False

In [ ]:
if not ONLY_PUBLIC:
    # !pip install ../input/segmentation-models-wheels/efficientnet_pytorch-0.6.3-py3-none-any.whl -q
    # !pip install ../input/segmentation-models-wheels/pretrainedmodels-0.7.4-py3-none-any.whl -q
    !pip install ../input/segmentation-models-wheels/timm-0.3.2-py3-none-any.whl -q
    # !pip install ../input/segmentationmodelspytorchmaster/segmentation_models.pytorch-master/ -q

In [ ]:
if not ONLY_PUBLIC:
    !conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
    !conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
    !conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
    !conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
    !conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
    !conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y

In [ ]:
if not ONLY_PUBLIC:    
    import os

    from PIL import Image
    import pandas as pd
    from tqdm.auto import tqdm

In [ ]:
if not ONLY_PUBLIC:
    df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
    if df.shape[0] == 247: #2477
        fast_sub = True
        fast_df = pd.DataFrame(([['00086460a852_study', 'negative 1 0 0 1 1'], 
                             ['000c9c05fd14_study', 'negative 1 0 0 1 1'], 
                             ['65761e66de9f_image', 'none 1 0 0 1 1'], 
                             ['51759b5579bc_image', 'none 1 0 0 1 1']]), 
                           columns=['id', 'PredictionString'])
    else:
        fast_sub = False


# .dcm to .png

In [ ]:
if not ONLY_PUBLIC:
    
    import numpy as np
    import pydicom
    from pydicom.pixel_data_handlers.util import apply_voi_lut

    def read_xray(path, voi_lut = True, fix_monochrome = True):
        # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
        dicom = pydicom.read_file(path)

        # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
        # "human-friendly" view
        if voi_lut:
            data = apply_voi_lut(dicom.pixel_array, dicom)
        else:
            data = dicom.pixel_array

        # depending on this value, X-ray may look inverted - fix that:
        if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
            data = np.amax(data) - data

        data = data - np.min(data)
        data = data / np.max(data)
        data = (data * 255).astype(np.uint8)

        return data

In [ ]:
if not ONLY_PUBLIC:    
    def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
        # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
        im = Image.fromarray(array)

        if keep_ratio:
            im.thumbnail((size, size), resample)
        else:
            im = im.resize((size, size), resample)

        return im

In [ ]:
if not ONLY_PUBLIC:
    split = 'test'
    save_dir = f'/kaggle/tmp/{split}/'
    os.makedirs(save_dir, exist_ok=True)

    save_dir_image = f'/kaggle/tmp/{split}/image/'
    os.makedirs(save_dir_image, exist_ok=True)

    save_dir_study = f'/kaggle/tmp/{split}/study/'
    os.makedirs(save_dir_study, exist_ok=True)

    image_id = []
    dim0 = []
    dim1 = []
    splits = []
    study_id = []

    if fast_sub:
        xray = read_xray('../input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm')
        im = resize(xray, size=600)  
        study = '00086460a852' + '_study.png'
        im.save(os.path.join(save_dir_study, study))
        xray = read_xray('../input/siim-covid19-detection/train/000c9c05fd14/e555410bd2cd/51759b5579bc.dcm')
        im = resize(xray, size=600)  
        study = '000c9c05fd14' + '_study.png'
        im.save(os.path.join(save_dir_study, study))
        study_id.append('00086460a852')
        study_id.append('000c9c05fd14')
    else:   
        for dirname, _, filenames in tqdm(os.walk(f'../input/siim-covid19-detection/{split}')):
            for file in filenames:
                # set keep_ratio=True to have original aspect ratio
                xray = read_xray(os.path.join(dirname, file))
                im_s = resize(xray, size=600)
                im_i = resize(xray, size=512)
                study = dirname.split('/')[-2] + '_study.png'
                im_s.save(os.path.join(save_dir_study, study))
                im_i.save(os.path.join(save_dir_image, file.replace('.dcm', '_image.png')))
                image_id.append(file.replace('.dcm', ''))
                study_id.append(dirname.split('/')[-2])
                dim0.append(xray.shape[0])
                dim1.append(xray.shape[1])
                splits.append(split)
            

In [ ]:
if not ONLY_PUBLIC:
    if fast_sub:
        xray = read_xray('../input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm')
        im = resize(xray, size=512)  
        im.save(os.path.join(save_dir_image,'65761e66de9f_image.png'))
        image_id.append('65761e66de9f.dcm'.replace('.dcm', ''))
        dim0.append(xray.shape[0])
        dim1.append(xray.shape[1])
        splits.append(split)
        xray = read_xray('../input/siim-covid19-detection/train/000c9c05fd14/e555410bd2cd/51759b5579bc.dcm')
        im = resize(xray, size=512)  
        im.save(os.path.join(save_dir_image, '51759b5579bc_image.png'))
        image_id.append('51759b5579bc.dcm'.replace('.dcm', ''))
        dim0.append(xray.shape[0])
        dim1.append(xray.shape[1])
        splits.append(split)

    meta = pd.DataFrame.from_dict({'image_id': image_id, 'dim0': dim0, 'dim1': dim1, 'split': splits, 'study_id': study_id})

# study predict

In [ ]:
if not ONLY_PUBLIC:
    import numpy as np 
    import pandas as pd
    if fast_sub:
        df = fast_df.copy()
    else:
        df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
    id_laststr_list  = []
    for i in range(df.shape[0]):
        id_laststr_list.append(df.loc[i,'id'][-1])
    df['id_last_str'] = id_laststr_list

    study_len = df[df['id_last_str'] == 'y'].shape[0]

In [ ]:
if not ONLY_PUBLIC:
    !pip install /kaggle/input/kerasapplications -q
    !pip install /kaggle/input/efficientnet-keras-source-code/ -q --no-deps

    import os

    import efficientnet.tfkeras as efn
    import numpy as np
    import pandas as pd
    import tensorflow as tf

    def auto_select_accelerator():
        try:
            tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            strategy = tf.distribute.experimental.TPUStrategy(tpu)
            print("Running on TPU:", tpu.master())
        except ValueError:
            strategy = tf.distribute.get_strategy()
        print(f"Running on {strategy.num_replicas_in_sync} replicas")

        return strategy


    def build_decoder(with_labels=True, target_size=(300, 300), ext='jpg'):
        def decode(path):
            file_bytes = tf.io.read_file(path)
            if ext == 'png':
                img = tf.image.decode_png(file_bytes, channels=3)
            elif ext in ['jpg', 'jpeg']:
                img = tf.image.decode_jpeg(file_bytes, channels=3)
            else:
                raise ValueError("Image extension not supported")

            img = tf.cast(img, tf.float32) / 255.0
            img = tf.image.resize(img, target_size)

            return img

        def decode_with_labels(path, label):
            return decode(path), label

        return decode_with_labels if with_labels else decode


    def build_augmenter(with_labels=True):
        def augment(img):
            img = tf.image.random_flip_left_right(img)
            img = tf.image.random_flip_up_down(img)
            return img

        def augment_with_labels(img, label):
            return augment(img), label

        return augment_with_labels if with_labels else augment


    def build_dataset(paths, labels=None, bsize=32, cache=True,
                      decode_fn=None, augment_fn=None,
                      augment=True, repeat=True, shuffle=1024, 
                      cache_dir=""):
        if cache_dir != "" and cache is True:
            os.makedirs(cache_dir, exist_ok=True)

        if decode_fn is None:
            decode_fn = build_decoder(labels is not None)

        if augment_fn is None:
            augment_fn = build_augmenter(labels is not None)

        AUTO = tf.data.experimental.AUTOTUNE
        slices = paths if labels is None else (paths, labels)

        dset = tf.data.Dataset.from_tensor_slices(slices)
        dset = dset.map(decode_fn, num_parallel_calls=AUTO)
        dset = dset.cache(cache_dir) if cache else dset
        dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
        dset = dset.repeat() if repeat else dset
        dset = dset.shuffle(shuffle) if shuffle else dset
        dset = dset.batch(bsize).prefetch(AUTO)

        return dset

    #COMPETITION_NAME = "siim-cov19-test-img512-study-600"
    strategy = auto_select_accelerator()
    BATCH_SIZE = strategy.num_replicas_in_sync * 16

    IMSIZE = (224, 240, 260, 300, 380, 456, 528, 600, 512)

    #load_dir = f"/kaggle/input/{COMPETITION_NAME}/"
    if fast_sub:
        sub_df = fast_df.copy()
    else:
        sub_df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
    sub_df = sub_df[:study_len]
    test_paths = f'/kaggle/tmp/{split}/study/' + sub_df['id'] +'.png'

    sub_df['negative'] = 0
    sub_df['typical'] = 0
    sub_df['indeterminate'] = 0
    sub_df['atypical'] = 0


    label_cols = sub_df.columns[2:]

    test_decoder = build_decoder(with_labels=False, target_size=(IMSIZE[7], IMSIZE[7]), ext='png')
    dtest = build_dataset(
        test_paths, bsize=BATCH_SIZE, repeat=False, 
        shuffle=False, augment=False, cache=False,
        decode_fn=test_decoder
    )

    with strategy.scope():

        models = []

        models0 = tf.keras.models.load_model(
            '../input/siim-covid19-efnb7-train-study/model0.h5'
        )
        models1 = tf.keras.models.load_model(
            '../input/siim-covid19-efnb7-train-study/model1.h5'
        )
        models2 = tf.keras.models.load_model(
            '../input/siim-covid19-efnb7-train-study/model2.h5'
        )
        models3 = tf.keras.models.load_model(
            '../input/siim-covid19-efnb7-train-study/model3.h5'
        )
        models4 = tf.keras.models.load_model(
            '../input/siim-covid19-efnb7-train-study/model4.h5'
        )

        models.append(models0)
        models.append(models1)
        models.append(models2)
        models.append(models3)
        models.append(models4)




    sub_df[label_cols] = sum([model.predict(dtest, verbose=1) for model in models]) / len(models)
    tf.keras.backend.clear_session()

# 2 class

In [ ]:
if not ONLY_PUBLIC:
    if fast_sub:
        sub_df_2 = fast_df.copy()
    else:
        sub_df_2 = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
    sub_df_2 = sub_df_2[study_len:]
    test_paths = f'/kaggle/tmp/{split}/image/' + sub_df_2['id'] +'.png'
    sub_df_2['none'] = 0

    label_cols = sub_df_2.columns[2]

    test_decoder = build_decoder(with_labels=False, target_size=(IMSIZE[8], IMSIZE[8]), ext='png')
    dtest = build_dataset(
        test_paths, bsize=BATCH_SIZE, repeat=False, 
        shuffle=False, augment=False, cache=False,
        decode_fn=test_decoder
    )

    with strategy.scope():

        models = []

        models0 = tf.keras.models.load_model(
            '../input/siim-covid19-efnb7-train-fold0-5-2class/model0.h5'
        )
        models1 = tf.keras.models.load_model(
            '../input/siim-covid19-efnb7-train-fold0-5-2class/model1.h5'
        )
        models2 = tf.keras.models.load_model(
            '../input/siim-covid19-efnb7-train-fold0-5-2class/model2.h5'
        )
        models3 = tf.keras.models.load_model(
            '../input/siim-covid19-efnb7-train-fold0-5-2class/model3.h5'
        )
        models4 = tf.keras.models.load_model(
            '../input/siim-covid19-efnb7-train-fold0-5-2class/model4.h5'
        )

        models.append(models0)
        models.append(models1)
        models.append(models2)
        models.append(models3)
        models.append(models4)




    sub_df_2[label_cols] = sum([model.predict(dtest, verbose=1) for model in models]) / len(models)
    df_2class = sub_df_2.reset_index(drop=True)

In [ ]:
if not ONLY_PUBLIC:
    del models
    del models0, models1, models2, models3, models4
    from numba import cuda
    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

In [ ]:
if not ONLY_PUBLIC:   
    import sys
    sys.path.append("../input/timm-pytorch-image-models/pytorch-image-models-master")
    import torch
    from torch.utils.data import DataLoader, Dataset
    import torch.nn as nn
    import torch.nn.functional as F
    from torchvision import transforms as T
    from torchvision.transforms import functional as FV
    #import segmentation_models_pytorch as smp
    import torchvision
    import albumentations, cv2
    import timm
    from tqdm import tqdm

    label_cols = sub_df.columns[2:]

    def get_path(idx):
        if not ONLY_PUBLIC:
            return f'/kaggle/tmp/{split}/study/' + idx +'.png'

        return TEST_PATH + idx +'.png'


    class SIIMDataset(Dataset):

        def __init__(self, df, transforms=None, subset="train", sat=True):

            super().__init__()
            self.df = df
            self.transforms = transforms
            self.subset = subset
            self.sat = sat

            if self.subset == "train":
                self.df = self.df[~self.df["is_valid"]]
            elif self.subset == "valid":
                self.df = self.df[self.df["is_valid"]]

        def __len__(self):
            return len(self.df)

        def __getitem__(self, idx):

            row = self.df.iloc[idx]
            img = cv2.imread(get_path(row.id))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            if self.sat:
                img = T.functional.to_pil_image(img)
                img = T.functional.adjust_saturation(img, 2)
                img = np.array(img)

            if self.transforms is not None:
                img = self.transforms(image=img)["image"]

            img = FV.to_tensor(img)

            if self.subset == "test":
                return img
            else:  
                label = row[cls].astype('int').values
                label = torch.as_tensor(label)   
                return img, label.float()

    def get_transform(image_size):

        transforms_test = albumentations.Compose([
        albumentations.Resize(image_size, image_size),
        #albumentations.HueSaturationValue(p=1, hue_shift_limit=0, sat_shift_limit=(0.15,0.16), val_shift_limit=0),
        albumentations.Normalize()])

        return transforms_test


    # def get_Unet(encoder_name, n_classes, pretrained=True):

    #     aux_params=dict(
    #     pooling='avg',             # one of 'avg', 'max'
    #     dropout=0.5,               # dropout ratio, default is None
    #     activation=None,      # activation function, default is None
    #     classes=n_classes,                 # define number of output labels
    #     )

    #     if pretrained:
    #         weights='imagenet'
    #     else:
    #         weights=None

    #     model =  smp.UnetPlusPlus(
    #                  encoder_name=encoder_name,
    #                  encoder_weights=weights,
    #                  in_channels=3,
    #                  classes=1, 
    #                 aux_params=aux_params)
    #     return model

    # class CustomModelSeg(nn.Module):

    #     def __init__(self, backbone, pretrained=False):
    #         super().__init__()
    #         self.backbone = backbone
    #         self.unet = get_Unet(self.backbone, 4, pretrained=pretrained)

    #     def forward(self, x, train=True):

    #         if train:
    #             return self.unet(x)

    #         e = self.unet.encoder(x)[-1]
    #         return self.unet.classification_head(e)

    class CustomModel(nn.Module):

        def __init__(self, backbone, pretrained=False):
            super().__init__()
            self.backbone = backbone
            self.model = timm.create_model(backbone, pretrained=pretrained)

            if "res" in self.backbone:
                self.n_features = self.model.fc.in_features
                self.model.fc = nn.Linear(self.n_features, 4)

            elif "vit" in self.backbone:
                self.n_features = self.head.fc.in_features
                self.head.fc = nn.Linear(self.n_features, 4)

            elif "nfnet" in self.backbone:
                self.n_features = self.model.head.fc.in_features
                self.model.head.fc = nn.Linear(self.n_features, 4)

            elif "ception" in self.backbone:
                self.n_features = self.model.last_linear.in_features
                self.model.last_linear = nn.Linear(self.n_features, 4)

            else:
                self.n_features = self.model.classifier.in_features
                self.model.classifier = nn.Linear(self.n_features, 4)


        def forward(self, x):
            h = self.model(x)
            return h

    class Model(nn.Module):

        def __init__(self, backbone, num_classes, dropout=0.5, pretrained=True):
            super(Model, self).__init__()

            self.backbone = backbone
            self.num_classes = num_classes
            self.base = timm.create_model(self.backbone, pretrained=pretrained)
            self.pooling = nn.AdaptiveAvgPool2d(1)
            self.dropout = nn.Dropout(dropout)

            if "res" in self.backbone:
                self.myfc = nn.Linear(self.base.fc.in_features, self.num_classes)
                self.base.fc = nn.Identity()

            elif "vit" in self.backbone:
                self.myfc = nn.Linear(self.base.head.in_features, self.num_classes)
                self.base.head = nn.Identity()

            elif "ception" in self.backbone:
                self.myfc = nn.Linear(self.base.last_linear.in_features, self.num_classes)
                self.base.global_pool = nn.Identity()
                self.base.last_linear = nn.Identity()

            elif "efficient" in self.backbone:
                self.myfc = nn.Linear(self.base.classifier.in_features, self.num_classes)
                self.base.classifier = nn.Identity()

            else:
                self.myfc = nn.Linear(self.base.classifier.in_features, self.num_classes)
                self.base.global_pool = nn.Identity()
                self.base.classifier = nn.Identity()

        def extract(self, x):
            return self.base(x)

        def forward(self, x):

            if "efficient" in self.backbone:
                x = self.extract(x)
                h = self.myfc(self.dropout(x))
                return h
            else:
                bs = x.size(0)
                features = self.base(x)
                pooled_features = self.pooling(features).view(bs, -1)
                output = self.myfc(pooled_features)
                return output

    def test_epoch(model, loader, bb=False):

        model.eval()
        PREDS = []

        with torch.no_grad():
            for data in tqdm(loader):
                data = data.to(device)
                if bb:
                    logits = model(data, False)
                else:
                    logits = model(data)
                act = nn.Softmax(dim=1)
                PREDS.append(act(logits.cpu()).numpy())

        return np.concatenate(np.array(PREDS))

    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    ####################################################################################################
    BASE = True

    if BASE:

        FOLDS = 5
        image_sizes = [512]*FOLDS
        batch_sizes = [16]*FOLDS

        backbone = ['tf_efficientnet_b5',
                    'tf_efficientnet_b5', 
                    'tf_efficientnet_b5',
                    'tf_efficientnet_b5',
                    'tf_efficientnet_b5']

        pre_trained_weights = ["../input/covid-new-fast/tf_efficientnet_b5_fold0_best_score.pth",
                               "../input/covid-new-fast/tf_efficientnet_b5_fold1_best_score.pth",
                               "../input/covid-new-fast/tf_efficientnet_b5_fold2_best_score.pth",
                               "../input/covid-new-fast/tf_efficientnet_b5_fold3_best_score.pth",
                               "../input/covid-new-fast/tf_efficientnet_b5_fold4_best_score.pth"]

        WGTS = [0.2]*FOLDS
        PREDS_1 = np.zeros(shape=(len(sub_df), 4))

        for fold in range(FOLDS):  

            transforms_test = get_transform(image_sizes[fold])

            dataset_test = SIIMDataset(sub_df, subset='test', transforms=transforms_test)

            test_loader = DataLoader(dataset_test, 
                                       batch_size=batch_sizes[fold], 
                                       shuffle=False, 
                                       num_workers=2)

            model = CustomModel(backbone=backbone[fold])

            if pre_trained_weights[fold] is not None:
                print("Loading Pretrained Weights.........")
                model.load_state_dict(torch.load(pre_trained_weights[fold])['model'])

            model = model.to(device)

            PREDS_1 += test_epoch(model, test_loader) * WGTS[fold]

    ####################################################################################################

    SEG = False    

    if SEG:

        FOLDS = 5
        image_sizes = [512]*FOLDS
        batch_sizes = [16]*FOLDS

        backbone = ['efficientnet-b3',
                    'efficientnet-b3', 
                    'efficientnet-b3',
                    'efficientnet-b3',
                    'efficientnet-b3']

        pre_trained_weights = ["../input/segmodelsuk/efficientnet-b3_fold0_best_loss.pth",
                               "../input/segmodelsuk/efficientnet-b3_fold1_best_loss.pth",
                               "../input/segmodelsuk/efficientnet-b3_fold2_best_loss.pth",
                               "../input/segmodelsuk/efficientnet-b3_fold3_best_loss.pth",
                               "../input/segmodelsuk/efficientnet-b3_fold4_best_loss.pth"]


        WGTS = [0.2]*FOLDS
        PREDS_SEG = np.zeros(shape=(len(sub_df), 4))
        device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

        for fold in range(FOLDS):  

            transforms_test = get_transform(image_sizes[fold])

            dataset_test = SIIMDataset(sub_df, subset='test', transforms=transforms_test)

            test_loader = DataLoader(dataset_test, 
                                       batch_size=batch_sizes[fold], 
                                       shuffle=False, 
                                       num_workers=2)

            model = CustomModelSeg(backbone=backbone[fold])

            if pre_trained_weights[fold] is not None:
                print("Loading Pretrained Weights.........")
                model.load_state_dict(torch.load(pre_trained_weights[fold], map_location=device)['model'])

            model = model.to(device)

            PREDS_SEG += test_epoch(model, test_loader, True) * WGTS[fold]

    #####################################################################################################    
    FIRST = True

    if FIRST:

        PREDS_2 = np.zeros(shape=(len(sub_df), 4))
        FOLDS = 3
        image_sizes = [320, 320, 224]
        batch_sizes = [32,32,32]

        backbone = ['densenet169',
                    'inception_v4', 
                    'tf_efficientnet_b5_ns']

        pre_trained_weights = ["../input/siimcovidpytorchdataset/models/densenet169_320_lr3e5_bs32_8epo_best_fold0.pth",
                               "../input/siimcovidpytorchdataset/models/inception_v4_320_lr3e5_bs32_8epo_best_fold1.pth",
                               "../input/siimcovidptmodels/tf_efficientnet_b5_ns_224_lr3e4_bs32_10epo_best_fold1.pth"]

        WGTS = [0.1, 0.2, 0.7]
        device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

        for fold in range(FOLDS): 

            transforms_test = get_transform(image_sizes[fold])

            dataset_test = SIIMDataset(sub_df, subset='test', transforms=transforms_test, sat=False)

            test_loader = DataLoader(dataset_test, 
                                       batch_size=batch_sizes[fold], 
                                       shuffle=False, 
                                       num_workers=2)

            model = Model(backbone=backbone[fold], num_classes=4, pretrained=False)

            if pre_trained_weights[fold] is not None:
                print("Loading Pretrained Weights.........")
                model.load_state_dict(torch.load(pre_trained_weights[fold]))

            model = model.to(device)

            P = test_epoch(model, test_loader) * WGTS[fold]

            if fold != 2:
                temp = P[:,0].copy()
                P[:,0] = P[:,1]
                P[:,1] = temp

            PREDS_2 += P

    ####################################################################################################

    full_preds = 0.5*PREDS_1 + 0.5*PREDS_2
    sub_df[label_cols] = 0.3*sub_df[label_cols] + 0.7*full_preds

In [ ]:
if not ONLY_PUBLIC:
    sub_df.columns = ['id', 'PredictionString1', 'negative', 'typical', 'indeterminate', 'atypical']
    df = pd.merge(df, sub_df, on = 'id', how = 'left')

    for i in range(study_len):
        negative = df.loc[i,'negative']
        typical = df.loc[i,'typical']
        indeterminate = df.loc[i,'indeterminate']
        atypical = df.loc[i,'atypical']
        df.loc[i, 'PredictionString'] = f'negative {negative} 0 0 1 1 typical {typical} 0 0 1 1 indeterminate {indeterminate} 0 0 1 1 atypical {atypical} 0 0 1 1'

    df_study = df[['id', 'PredictionString']]

In [ ]:
if not ONLY_PUBLIC:
    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

# yolov5 predict

In [ ]:
if not ONLY_PUBLIC:
    import numpy as np, pandas as pd
    from glob import glob
    import shutil, os
    import matplotlib.pyplot as plt
    from sklearn.model_selection import GroupKFold
    from tqdm.notebook import tqdm
    import seaborn as sns
    import torch

In [ ]:
if not ONLY_PUBLIC:
    meta = meta[meta['split'] == 'test']
    if fast_sub:
        test_df = fast_df.copy()
    else:
        test_df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
    test_df = df[study_len:].reset_index(drop=True) 
    meta['image_id'] = meta['image_id'] + '_image'
    meta['study_id'] = meta['study_id'] + '_study'
    meta.columns = ['id', 'dim0', 'dim1', 'split', 'study_id']
    test_df = pd.merge(test_df, meta, on = 'id', how = 'left')


In [ ]:
if not ONLY_PUBLIC:    
    !pip install ../input/ensembleboxes-offline-package/ensemble_boxes-1.0.4-py3-none-any.whl -q

In [ ]:
if not ONLY_PUBLIC:
    #INPUT_PATH = '../input/siim-covid19-detection/'
    OUTPUT_PATH = '/kaggle/working/'
    SUBM_PATH = '/kaggle/working/'

In [ ]:
if not ONLY_PUBLIC:    
    from ensemble_boxes import weighted_boxes_fusion


    def ensemble(
        subm_list,
        iou_same=0.5,
        out_path=None,
        skip_box_thr=0.00000001,
    ):
        sizes = get_train_test_image_sizes()
        preds = []
        weights = []
        checker = None
        for df, weight in subm_list:
            s = df.copy()
            s.sort_values('id', inplace=True)
            s.reset_index(drop=True, inplace=True)
            ids = s['id']

            if checker:
                if tuple(ids) != checker:
                    print(set(checker) - set(ids))
                    print('Different IDS!', len(tuple(ids)), path)
                    exit()
            else:
                checker = tuple(ids)

            preds.append(s['PredictionString'].values)
            weights.append(weight)

        if out_path is None:
            out_path = SUBM_PATH + 'ensemble_iou_{}.csv'.format(iou_same)
            
        out = open(out_path, 'w')
        out.write('id,PredictionString\n')
        for j, id in enumerate(list(checker)):
            #print(id)
            boxes_list = []
            scores_list = []
            labels_list = []
            empty = True
            for i in range(len(preds)):
                boxes = []
                scores = []
                labels = []
                #print(preds[i])
                p1 = preds[i][j]
                if str(p1) != 'none 1 0 0 1 1':
                    arr = p1.strip().split(' ')
                    for k in range(0, len(arr), 6):
                        cls = int(arr[k])
                        prob = float(arr[k + 1])
                        x1 = float(arr[k + 2]) / sizes[id][1]
                        y1 = float(arr[k + 3]) / sizes[id][0]
                        x2 = float(arr[k + 4]) / sizes[id][1]
                        y2 = float(arr[k + 5]) / sizes[id][0]
                        boxes.append([x1, y1, x2, y2])
                        scores.append(prob)
                        labels.append(cls)
                    

                boxes_list.append(boxes)
                scores_list.append(scores)
                labels_list.append(labels)

            boxes, scores, labels = weighted_boxes_fusion(
                boxes_list,
                scores_list,
                labels_list,
                iou_thr=iou_same,
                skip_box_thr=skip_box_thr,
                weights=weights,
                allows_overflow=True
            )
            #print(len(boxes), len(labels), len(scores))
            #prior_2 = 1 - df_2class.query(f'id == "{id}"')["none"]
            #s_id = test_df.query(f'id == "{id}"')["study_id"]
            #prior_st = 1 - sub_df.query(f'id == "{s_id}"')["negative"]
            #prior = prior_2 #*0.5 + prior_st*0.5
            if len(boxes) == 0:
                out.write('{},none 1 0 0 1 1\n'.format(id, ))
            else:
                final_str = ''
                for i in range(len(boxes)):
                    cls = int(labels[i])
                    prob = float(scores[i])
                    x1 = int(boxes[i][0] * sizes[id][1])
                    y1 = int(boxes[i][1] * sizes[id][0])
                    x2 = int(boxes[i][2] * sizes[id][1])
                    y2 = int(boxes[i][3] * sizes[id][0])
                    final_str += '{} {} {} {} {} {} '.format(cls, prob, x1, y1, x2, y2)
                out.write('{},{}\n'.format(id, final_str.strip()))

        out.close()
        return out_path


    # def get_test_from_subm_list(subm_list):
    #     out = []
    #     for s, w in subm_list:
    #         s1 = s.replace('_train', '_test')
    #         out.append((s1, w))
    #     return out

    def get_train_test_image_sizes():
        sizes = dict()
        sizes_df = test_df.copy()
        for index, row in sizes_df.iterrows():
            sizes[row['id']] = (row['dim0'], row['dim1'])
        return sizes


    def ensemble_experiment_v4_yolo(preds_df_all):

        subm_list = [
            (preds_df_all[0], 1),
            (preds_df_all[1], 1),
            (preds_df_all[2], 1),
            (preds_df_all[3], 1),
            (preds_df_all[4], 1)
        ]
        #subm_list_test = get_test_from_subm_list(subm_list)

        best_iou = 0.6
        #out_path = SUBM_PATH + 'ensemble_yolo_standard.csv'.format(len(subm_list_test), best_iou)
        predictions = ensemble(subm_list, best_iou)

        return predictions

In [ ]:
if not ONLY_PUBLIC:
    dim = 512 #1024, 256, 'original'
    test_dir = f'/kaggle/tmp/{split}/image'
    weights_dir = ['/kaggle/input/siim-cov19-yolov5-train/yolov5/runs/train/exp/weights/best.pt',
                   '/kaggle/input/covid-yolo-fold-1/yolov5/runs/train/exp/weights/best.pt',
                   '/kaggle/input/covid-yolo-fold-2/yolov5/runs/train/exp/weights/best.pt',
                   '/kaggle/input/covid-yolo-fold-3/yolov5/runs/train/exp/weights/best.pt',
                   '/kaggle/input/covid-yolo-fold-4/yolov5/runs/train/exp/weights/best.pt']

    shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', '/kaggle/working/yolov5')
    os.chdir('/kaggle/working/yolov5') # install dependencies

    import torch
    #from IPython.display import Image, clear_output  # to display images

    #clear_output()
    #print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

    #fold 0
    wts = weights_dir[0]
    !python detect.py --weights $wts\
    --img 512\
    --conf 0.001\
    --iou 0.5\
    --source $test_dir\
    --project /kaggle/working/yolov5_fold0/ --name test_iou_0.5_0.001\
    --save-txt --save-conf --exist-ok

    #fold 1
    wts = weights_dir[1]
    !python detect.py --weights $wts\
    --img 512\
    --conf 0.001\
    --iou 0.5\
    --source $test_dir\
    --project /kaggle/working/yolov5_fold1/ --name test_iou_0.5_0.001\
    --save-txt --save-conf --exist-ok

    #fold 2
    wts = weights_dir[2]
    !python detect.py --weights $wts\
    --img 512\
    --conf 0.001\
    --iou 0.5\
    --source $test_dir\
    --project /kaggle/working/yolov5_fold2/ --name test_iou_0.5_0.001\
    --save-txt --save-conf --exist-ok
    
    #fold 3
    wts = weights_dir[3]
    !python detect.py --weights $wts\
    --img 512\
    --conf 0.001\
    --iou 0.5\
    --source $test_dir\
    --project /kaggle/working/yolov5_fold3/ --name test_iou_0.5_0.001\
    --save-txt --save-conf --exist-ok
    
    #fold 4
    wts = weights_dir[4]
    !python detect.py --weights $wts\
    --img 512\
    --conf 0.001\
    --iou 0.5\
    --source $test_dir\
    --project /kaggle/working/yolov5_fold4/ --name test_iou_0.5_0.001\
    --save-txt --save-conf --exist-ok


    def yolo2voc(image_height, image_width, bboxes):
        """
        yolo => [xmid, ymid, w, h] (normalized)
        voc  => [x1, y1, x2, y1]

        """ 
        bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int

        bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
        bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height

        bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
        bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]

        return bboxes

    preds_df_all = []

    for fold in range(5):

        image_ids = []
        PredictionStrings = []

        for file_path in tqdm(glob('/kaggle/working/yolov5_fold{}/test_iou_0.5_0.001/labels/*.txt'.format(fold))):
            image_id = file_path.split('/')[-1].split('.')[0]
            w, h = test_df.loc[test_df.id==image_id,['dim1', 'dim0']].values[0]
            f = open(file_path, 'r')
            data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
            data = data[:, [0, 5, 1, 2, 3, 4]]
            bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 12).astype(str))
            for idx in range(len(bboxes)):
                bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
            image_ids.append(image_id)
            PredictionStrings.append(' '.join(bboxes))
            
        full_df = test_df.copy().drop(['PredictionString'], axis=1)
        preds_df = pd.DataFrame({'id':image_ids,'PredictionString':PredictionStrings})
        preds_df_full = pd.merge(full_df, preds_df, on = 'id', how = 'left').fillna("none 1 0 0 1 1")
        preds_df_all.append(preds_df_full)

    out_path = ensemble_experiment_v4_yolo(preds_df_all)

In [ ]:
if not ONLY_PUBLIC:    
#     test_df = test_df.drop(['PredictionString'], axis=1)
#     pred_df = pd.read_csv(out_path)
#     sub_df = pd.merge(test_df, pred_df, on = 'id', how = 'left').fillna("none 1 0 0 1 1")
#     sub_df = sub_df[['id', 'PredictionString']]
    sub_df = pd.read_csv(out_path)[['id', 'PredictionString']]
    for i in range(sub_df.shape[0]):
        if sub_df.loc[i,'PredictionString'] == "none 1 0 0 1 1":
            continue
        sub_df_split = sub_df.loc[i,'PredictionString'].split()
        sub_df_list = []
        for j in range(int(len(sub_df_split) / 6)):
            sub_df_list.append('opacity')
            sub_df_list.append(sub_df_split[6 * j + 1])
            sub_df_list.append(sub_df_split[6 * j + 2])
            sub_df_list.append(sub_df_split[6 * j + 3])
            sub_df_list.append(sub_df_split[6 * j + 4])
            sub_df_list.append(sub_df_split[6 * j + 5])
        sub_df.loc[i,'PredictionString'] = ' '.join(sub_df_list)
    sub_df['none'] = df_2class['none'] 
    for i in range(sub_df.shape[0]):
        if sub_df.loc[i,'PredictionString'] != 'none 1 0 0 1 1':
            sub_df.loc[i,'PredictionString'] = sub_df.loc[i,'PredictionString'] + ' none ' + \
            str(sub_df.loc[i,'none']) + ' 0 0 1 1'
    sub_df = sub_df[['id', 'PredictionString']]   
    df_study = df_study[:study_len]
    df_study = df_study.append(sub_df).reset_index(drop=True)
    df_study.to_csv('/kaggle/working/submission.csv',index = False)  
    shutil.rmtree('/kaggle/working/yolov5')

In [ ]:
if not ONLY_PUBLIC:
    !rm -rf /kaggle/working/yolov5_fold0
    !rm -rf /kaggle/working/yolov5_fold1
    !rm -rf /kaggle/working/yolov5_fold2
    !rm -rf /kaggle/working/yolov5_fold3
    !rm -rf /kaggle/working/yolov5_fold4
    os.remove('/kaggle/working/ensemble_iou_0.3.csv')

In [ ]:
if ONLY_PUBLIC:
    sub_df = pd.read_csv("../input/siim-covid19-detection/sample_submission.csv")
    pred_sub = pd.read_csv("../input/siim-covid19-detection/sample_submission.csv")[0:100]
    for i in range(len(sub_df)):
        if not sub_df.loc[i, "id"].isin(pred_sub["id"]): 
            continue
        sub_df.loc[i, "PredictionString"]=pred_sub.query(f'id == "{sub_df.loc[i, "id"]}"')["PredictionString"]        

In [ ]:
df_study